In [ ]:
import skel_features
from standard_transform import minnie_ds
import pandas as pd
import numpy as np
from meshparty import meshwork

from cloudpathlib import AnyPath as Path

In [ ]:
# skel_dir = Path('/Users/caseysm/Work/Code/Tasks/filequeue/output')
skel_dir = Path("gs://csm-skel/skel-test/")
feature_dir = 'features/minnie65_phase3_v1_spines'

height_bnds = [0, 850]
layer_bnds = np.load('/Users/caseysm/Work/Projects/MinnieColumn/data/layer_bounds_v3.npy')

In [ ]:
file_df = pd.read_feather(Path('~/Work/Code/Tasks/hydratedskeletons/column_task_sampler.feather'))

In [ ]:
success = skel_features.extraction.extract_features_mp(
    file_df['root_id'],
    skel_dir,
    height_bnds,
    minnie_ds,
    feature_dir,
    dist_bins=5,
    max_dist=200,
    nodes=20,
    rerun=False,
    model_config='minnie65_ds15_us0_bd0',
    synapse_transform='nm',
    # save_synapse_count=True,
)

In [ ]:
file_df['success'] = success

In [ ]:
file_df = file_df.query('success').reset_index()

In [ ]:
file_df.to_feather('v1300_axon_label.feather')

In [ ]:
file_df = pd.read_feather('v1300_axon_label.feather')

In [ ]:
# raw_df = skel_features.io_utils.load_features(file_df['root_id'], feature_dir)
raw_df = skel_features.io_utils.load_features(file_df['root_id'], feature_dir)

In [ ]:
ii, col = np.where(raw_df.isna())

In [ ]:
len(col)

In [ ]:
raw_df.to_feather('v1300_spine_pred_axon_label.feather')

In [ ]:
raw_df = pd.read_feather('v1300_spine_pred_axon_label.feather')

In [ ]:
from sklearn import preprocessing, decomposition

In [ ]:
ego_syn_pca = decomposition.SparsePCA(n_components=5)
ego_pproc = preprocessing.StandardScaler()

In [ ]:
pl_dat = np.vstack(raw_df["egocentric_bins"].values)
pl_dat_norm = pl_dat / np.atleast_2d(pl_dat.sum(axis=1)).T

In [ ]:
ego_rows, _ = np.where(np.isnan(np.vstack(raw_df['egocentric_bins'])))

In [ ]:
net_ego_syn = raw_df['egocentric_bins'].apply(lambda x: sum(x))
raw_df['net_ego_syn'] = net_ego_syn.replace({0: np.nan})

In [ ]:
(
    feat_df_alt,
    feat_cols,
    syn_pca,
    br_svd,
    keep_depth,
    ego_pca,
    model_dict,
) = skel_features.assembly.assemble_features_from_data(raw_df.dropna(), n_syn_comp=10, n_branch_comp=5, n_syn_ego=5, use_spines=True)

In [ ]:
feat_df_alt.shape

In [ ]:
feat_df_alt.to_feather('features/minnie65_phase3_v1_spines/feat_df_spine_axon_prediction.feather')

In [ ]:
import json
with open('features/minnie65_phase3_v1_spines/feat_cols_spine_axon_prediction.json', 'w') as f:
    json.dump(feat_cols, f)

In [ ]:
import json

In [ ]:
syn_counts = []
rids = []
for root_id in file_df['root_id']:
    fname = f'{feature_dir}/{root_id}_syn_count.json'
    try:
        with open(fname) as f:
            syn_count = json.load(f)
        syn_counts.append(syn_count)    
        rids.append(root_id)
    except:
        continue

axon_compartment_df = pd.DataFrame(
    {
        'root_id': rids,
        'syn_count': syn_counts,
    }
)

In [ ]:
axon_compartment_df = axon_compartment_df.explode('syn_count').reset_index(drop=True)

In [ ]:
axon_compartment_df.to_feather('v1300_axon_compartments.feather')

In [ ]:
blah = axon_compartment_df.groupby('root_id').transform('count')

In [ ]:
axon_compartment_df['axon_compartment'] = count_val

In [ ]:
axon_compartment_df

In [ ]:
nrn = meshwork.load_meshwork(filename=skel_dir / '864691135560849377.h5')

In [ ]:
nrn.anno.post_syn.df

In [ ]:
nrn.anno.post_syn.df

In [ ]:
import skel_features.skel_filtering


height_bounds = height_bnds
n_egocentric_bins = 14

depth_bins = skel_features.extraction.make_depth_bins(height_bounds)
egocentric_bins = skel_features.extraction.make_egocentric_bins(100, n_egocentric_bins)
radius_bins = np.arange(20, 300, 30)
radius_bin_width = 10

nrn = skel_features.io_utils.load_root_id(864691136674495623, skel_dir, peel_threshold=0.1)

feats = skel_features.extraction.extract_features_dict(
    nrn,
    radius_bins=radius_bins,
    radius_bin_width=radius_bin_width,
    depth_bins=depth_bins,
    egocentric_bins=egocentric_bins,
    sl_dataset=minnie_ds,
)

In [ ]:
feats['syn_count_depth_shaft_dendrite'] / (feats['syn_count_depth_dendrite']+ 0.00000001)

In [ ]:
feats['syn_count_depth_dendrite']